In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import httpx
from brokit.primitives.lm import LM, ModelType, ModelResponse, Usage, Message
from typing import List, Optional

class Llama(LM):
    def __init__(self, model_name: str, base_url:str = "http://localhost:11434", temperature:float=0.0, top_p:float=1.0, seed:int=55, **kwargs):
        super().__init__(model_name=model_name, model_type=ModelType.CHAT)
        self.base_url = base_url
        self.client = httpx.Client(timeout=60.0)  # Reusable client)
        self.model_params = {
            "temperature": temperature,
            "top_p": top_p,
            "seed": seed,
            **kwargs
        }

    def request(self, prompt:Optional[str]=None, messages:Optional[List[Message]]=None, **kwargs) -> dict:
        url = f"{self.base_url}/api/chat"
        params = {**self.model_params, **kwargs}
        if messages is not None:
            _messages = [msg.to_dict() if isinstance(msg, Message) else msg for msg in messages]
        else:
            _messages = [{"role": "user", "content": prompt}]
        response = self.client.post(
            url,
            json={
                "model": self.model_name,
                "messages": _messages,
                "stream": False,
                "options": {**params},
            }
        )                
        return response.json()

    def parse_response(self, original_response: dict) -> ModelResponse:
        message = original_response["message"]
        input_tokens = original_response.get("prompt_eval_count", 0)
        output_tokens = original_response.get("eval_count", 0)
        return ModelResponse(
            model_name=self.model_name,
            model_type=self.model_type,
            response=message["content"],
            usage=Usage(input_tokens=input_tokens, output_tokens=output_tokens),
            metadata=None
        )

In [3]:
llama = Llama(model_name="gemma3:12b")

In [4]:
llama.request(prompt="Hello, how are you?")

{'model': 'gemma3:12b',
 'created_at': '2026-02-01T15:43:44.0203002Z',
 'message': {'role': 'assistant',
  'content': "Hello! I'm doing well, thank you for asking! As an AI, I don't experience feelings like humans do, but I'm operating smoothly and ready to assist you. 😊 \n\nHow are *you* doing today?"},
 'done': True,
 'done_reason': 'stop',
 'total_duration': 988216500,
 'load_duration': 226956400,
 'prompt_eval_count': 15,
 'prompt_eval_duration': 68817600,
 'eval_count': 51,
 'eval_duration': 682263200}

In [5]:
llama(prompt="Hello, how are you?")

ModelResponse(model_name='gemma3:12b', model_type=<ModelType.CHAT: 'chat'>, response="Hello! I'm doing well, thank you for asking! As an AI, I don't experience feelings like humans do, but I'm operating smoothly and ready to assist you. 😊 \n\nHow are *you* doing today?", usage=Usage(input_tokens=15, output_tokens=51), response_ms=950.3395000028831, cached=False, metadata=None, request=None, parsed_response=None)

In [6]:
llama(prompt="Hello, how are you?")

ModelResponse(model_name='gemma3:12b', model_type=<ModelType.CHAT: 'chat'>, response="Hello! I'm doing well, thank you for asking! As an AI, I don't experience feelings like humans do, but I'm operating smoothly and ready to assist you. 😊 \n\nHow are *you* doing today?", usage=Usage(input_tokens=15, output_tokens=51), response_ms=950.3395000028831, cached=True, metadata=None, request=None, parsed_response=None)

In [7]:
from brokit.primitives.prompt import Image
doraemon1 = Image("https://aithailand.co.th/wp-content/uploads/2024/09/Doraemon-th_1630640671-1024x635.jpg")
nobita1 = Image("https://m.media-amazon.com/images/M/MV5BM2E5MDIwNjktOWQwYS00ODg5LTlhN2MtNzU1OTc3MDA0NjNmXkEyXkFqcGc@._V1_FMjpg_UX1000_.jpg")

In [8]:
message = Message(**{"role": "user", "content": "What color is Nobita's shirt?", "images": [doraemon1.to_base64(), nobita1.to_base64()]})
messages = [message]

In [9]:
message.to_dict()

{'role': 'user',
 'content': "What color is Nobita's shirt?",
 'images': ['...']}

In [10]:
response = llama.request(messages=messages)

In [11]:
print(llama.parse_response(response).response)

Based on the image, Nobita's shirt is **yellow**.
